In [2]:
import pandas as pd
import glob
import os

In [5]:
def load_asd_data_with_pandas(data_path: str) -> pd.DataFrame:
    """
    Загружает все parquet файлы из папки asd в один pandas DataFrame

    Args:
        data_path: путь к папке с данными

    Returns:
        pd.DataFrame: объединенный DataFrame со всеми данными
    """
    # Получаем все parquet файлы из папки
    parquet_files = glob.glob(os.path.join(data_path, "part-*.parquet"))

    if not parquet_files:
        raise ValueError(f"Не найдено parquet файлов в папке {data_path}")

    print(f"Найдено {len(parquet_files)} parquet файлов")

    # Загружаем все файлы в список DataFrame'ов
    dataframes = []
    for file_path in parquet_files:
        df = pd.read_parquet(file_path)
        dataframes.append(df)

    # Объединяем все DataFrame'ы в один
    combined_df = pd.concat(dataframes, ignore_index=True)

    print(f"Общий размер данных: {combined_df.shape}")
    print(f"Колонки: {list(combined_df.columns)}")

    return combined_df

# Загружаем данные
agab_df = load_asd_data_with_pandas('./asd')

Найдено 20 parquet файлов
Общий размер данных: (1227083, 11)
Колонки: ['dataset', 'heavy_sequence', 'light_sequence', 'scfv', 'affinity_type', 'affinity', 'antigen_sequence', 'confidence', 'nanobody', 'metadata', 'processed_measurement']


In [51]:
is_not_nanobody = agab_df['nanobody'] == False
is_high_confidence = agab_df['confidence'].isin(['high', 'very_high'])
is_scfv = agab_df['scfv'] == True
has_both_chains = (
    agab_df['light_sequence'].notna() 
    & agab_df['heavy_sequence'].notna()
    & (agab_df['light_sequence'] != '')
    & (agab_df['heavy_sequence'] != '')
)

agab_df = agab_df[is_not_nanobody & is_high_confidence & (is_scfv | has_both_chains)]
agab_df.shape

(823251, 11)

In [ ]:
# (порог, оператор): '<', '>', '=='
AFFINITY_THRESHOLDS = {
    'fuzzy': ('h', '=='),
    'bool': (1, '=='),
    'alphaseq': (2, '<'),
    '-log KD': (7, '>'),
    'kd': (100, '<'),
    'delta_g': (-9.5, '<'),
    'log_enrichment': (1, '>'),
    'elisa_mut_to_wt_ratio': (1, '>'),
    'ic_50': (100, '<'),
}

def apply_affinity_filter(df: pd.DataFrame, thresholds: dict) -> pd.DataFrame:
    masks = []
    for affinity_type, (threshold, op) in thresholds.items():
        type_mask = df['affinity_type'] == affinity_type
        if op == '==':
            mask = type_mask & (df['affinity'] == threshold)
        else:
            numeric_affinity = pd.to_numeric(df['affinity'], errors='coerce')
            if op == '<':
                mask = type_mask & (numeric_affinity < threshold)
            else:
                mask = type_mask & (numeric_affinity > threshold)
        masks.append(mask)

    if masks:
        combined_mask = masks[0]
        for mask in masks[1:]:
            combined_mask = combined_mask | mask
        return df[combined_mask]
    else:
        return df

print(f"После базовых фильтров: {agab_df.shape}")
print(f"Распределение по affinity_type до фильтрации по порогам:")
print(agab_df['affinity_type'].value_counts())

# Применяем фильтрацию по порогам аффиности
agab_df = apply_affinity_filter(agab_df, AFFINITY_THRESHOLDS)

print(f"\nПосле фильтрации по порогам аффинитета: {agab_df.shape}")
print(f"Распределение по affinity_type после фильтрации:")
print(agab_df['affinity_type'].value_counts())


После базовых фильтров: (823251, 11)
Распределение по affinity_type до фильтрации по порогам:
affinity_type
fuzzy                    524346
-log KD                  152401
alphaseq                 131645
kd                         6849
log_enrichment             3452
bool                       2870
ddg                         670
elisa_mut_to_wt_ratio       658
ic_50                       360
Name: count, dtype: int64

После фильтрации по порогам аффинитета: (324227, 11)
Распределение по affinity_type после фильтрации:
affinity_type
fuzzy                    172149
-log KD                   87733
alphaseq                  56068
kd                         6744
log_enrichment             1016
ic_50                       349
elisa_mut_to_wt_ratio       168
Name: count, dtype: int64


In [ ]:
print("=== АНАЛИЗ РАЗЛИЧИЙ В ПОЛЯХ ДЛЯ ДУБЛИКАТОВ ПО ПОСЛЕДОВАТЕЛЬНОСТЯМ (agab_df) ===\n")

# Поля для проверки
fields = [
    "dataset",
    "scfv",
    "affinity_type",
    "affinity",
    "confidence",
    "nanobody",
    "processed_measurement",
]

# --- Поиск групп дубликатов по heavy_sequence + light_sequence + antigen_sequence---
# Находим группы, где хотя бы 2 записи с одинаковой парой последовательностей
group_cols = ["heavy_sequence", "light_sequence", "antigen_sequence"]
grouped = agab_df.groupby(group_cols, sort=False)

# Маска строк, которые входят в группы размером > 1
dup_mask = grouped[group_cols[0]].transform("size") > 1
hl_duplicates = agab_df.loc[dup_mask].copy()

# Переиспользуем groupby только на дубликатах
dup_groups = hl_duplicates.groupby(group_cols, sort=False)
unique_groups_count = dup_groups.ngroups

print(f"Всего групп дубликатов: {unique_groups_count}\n")

# --- Анализ различий по полям ---

# Для каждой группы считаем, сколько уникальных значений в каждом поле
# dropna=False, чтобы различия NaN / не-NaN тоже учитывались
nunique_per_group = dup_groups[fields].nunique(dropna=False)

# Булева матрица: True, если в группе по полю есть различия
diff_mask = nunique_per_group > 1

# Есть ли вообще различия в группе
group_has_diffs = diff_mask.any(axis=1)

groups_with_diffs = int(group_has_diffs.sum())
groups_identical = int((~group_has_diffs).sum())
total = groups_with_diffs + groups_identical if (groups_with_diffs + groups_identical) > 0 else 1

print(f"Групп с различиями: {groups_with_diffs} ({groups_with_diffs / total * 100:.2f}%)")
print(f"Групп без различий: {groups_identical} ({groups_identical / total * 100:.2f}%)\n")

# Количество строк (записей) с различиями и без различий
rows_with_diffs = group_has_diffs[group_has_diffs].index  # индексы групп с различиями
rows_without_diffs = group_has_diffs[~group_has_diffs].index  # индексы групп без различий

num_rows_with_diffs = dup_groups.size().loc[rows_with_diffs].sum() if len(rows_with_diffs) > 0 else 0
num_rows_without_diffs = dup_groups.size().loc[rows_without_diffs].sum() if len(rows_without_diffs) > 0 else 0
rows_total = num_rows_with_diffs + num_rows_without_diffs if (num_rows_with_diffs + num_rows_without_diffs) > 0 else 1

print(f"Строк (записей) с различиями: {num_rows_with_diffs} ({num_rows_with_diffs / rows_total * 100:.2f}%)")
print(f"Строк (записей) без различий: {num_rows_without_diffs} ({num_rows_without_diffs / rows_total * 100:.2f}%)\n")

# Считаем, по скольким группам отличается каждое поле
field_diffs_counts = diff_mask.sum().sort_values(ascending=False)

if groups_with_diffs > 0 and not field_diffs_counts.empty:
    print("=== ПОЛЯ, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===\n")
    for field, count in field_diffs_counts.items():
        if count == 0:
            continue
        print(f"{field}: {count} групп ({count / groups_with_diffs * 100:.2f}%)")

=== АНАЛИЗ РАЗЛИЧИЙ В ПОЛЯХ ДЛЯ ДУБЛИКАТОВ ПО ПОСЛЕДОВАТЕЛЬНОСТЯМ (agab_df) ===

Всего групп дубликатов: 6052

Групп с различиями: 6049 (99.95%)
Групп без различий: 3 (0.05%)

Строк (записей) с различиями: 12662 (99.95%)
Строк (записей) без различий: 6 (0.05%)

=== ПОЛЯ, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===

affinity: 6049 групп (100.00%)


In [62]:
print("=== АНАЛИЗ РАЗБРОСА АФФИНИТЕТА В ГРУППАХ ДУБЛИКАТОВ ===\n")

# Работаем с копией дубликатов
analysis_df = hl_duplicates.copy()

# Преобразуем аффинитет в числа, нечисловые значения станут NaN
analysis_df['affinity_numeric'] = pd.to_numeric(analysis_df['affinity'], errors='coerce')

# Группируем
grouped_analysis = analysis_df.groupby(group_cols, sort=False)

# Считаем размах (max - min) и проверяем единственность типа аффинитета
agg_funcs = {
    'affinity_numeric': lambda x: x.max() - x.min(),
    'affinity_type': 'nunique'
}

group_stats = grouped_analysis.agg(agg_funcs)
group_stats.columns = ['affinity_range', 'affinity_type_count']

# Фильтруем группы, где affinity_numeric удалось посчитать (не NaN)
valid_stats = group_stats.dropna(subset=['affinity_range'])

print(f"Групп с валидными числовыми аффинитетами: {len(valid_stats)}")
print(f"Средний разброс аффинитета в группе: {valid_stats['affinity_range'].mean():.4f}")
print(f"Медианный разброс: {valid_stats['affinity_range'].median():.4f}")
print(f"Максимальный разброс: {valid_stats['affinity_range'].max():.4f}\n")

# Проверка на разные типы аффинитета в одной группе
mixed_types = valid_stats[valid_stats['affinity_type_count'] > 1]
if not mixed_types.empty:
    print(f"ВНИМАНИЕ: В {len(mixed_types)} группах смешаны разные типы аффинитета (сравнение может быть некорректным).")
else:
    print("Типы аффинитета внутри каждой группы совпадают (корректно сравнивать числа).\n")

# Топ-5 групп с самым большим разбросом
print("=== ТОП-5 ГРУПП С САМЫМ БОЛЬШИМ РАЗБРОСОМ АФФИНИТЕТА ===")
top_diff_groups = valid_stats.nlargest(5, 'affinity_range')

for idx, (indices, row) in enumerate(top_diff_groups.iterrows()):
    heavy, light, antigen = indices
    print(f"\nГруппа {idx+1} (разброс {row['affinity_range']:.4f}):")
    # Получаем строки этой группы
    group_rows = analysis_df[
        (analysis_df['heavy_sequence'] == heavy) & 
        (analysis_df['light_sequence'] == light) &
        (analysis_df['antigen_sequence'] == antigen)
    ]
    display(group_rows[['dataset', 'affinity_type', 'affinity', 'confidence', 'processed_measurement']])

=== АНАЛИЗ РАЗБРОСА АФФИНИТЕТА В ГРУППАХ ДУБЛИКАТОВ ===

Групп с валидными числовыми аффинитетами: 6052
Средний разброс аффинитета в группе: 1.2487
Медианный разброс: 1.3385
Максимальный разброс: 2.8435

Типы аффинитета внутри каждой группы совпадают (корректно сравнивать числа).

=== ТОП-5 ГРУПП С САМЫМ БОЛЬШИМ РАЗБРОСОМ АФФИНИТЕТА ===

Группа 1 (разброс 2.8435):


,dataset,affinity_type,affinity,confidence,processed_measurement
279665,alphaseq,alphaseq,1.0284266312344492,high,1.1118588157602758
279666,alphaseq,alphaseq,-1.0209442312708925,high,1.1118588157602758
279667,alphaseq,alphaseq,1.8225332451339469,high,1.1118588157602758



Группа 2 (разброс 2.6122):


,dataset,affinity_type,affinity,confidence,processed_measurement
714760,abbd,-log KD,9.62101875525756,high,8.314932834001986
714761,abbd,-log KD,7.008846912746411,high,8.314932834001986



Группа 3 (разброс 2.6086):


,dataset,affinity_type,affinity,confidence,processed_measurement
38065,abbd,-log KD,9.61745476693648,high,8.313173132795084
38066,abbd,-log KD,7.008891498653689,high,8.313173132795084



Группа 4 (разброс 2.5932):


,dataset,affinity_type,affinity,confidence,processed_measurement
1146087,abbd,-log KD,9.593389009569856,high,8.296774595329161
1146088,abbd,-log KD,7.000160181088464,high,8.296774595329161



Группа 5 (разброс 2.5851):


,dataset,affinity_type,affinity,confidence,processed_measurement
1207842,abbd,-log KD,9.591688775955054,high,8.299143333258883
1207843,abbd,-log KD,7.006597890562709,high,8.299143333258883


In [63]:
# Удаляем дубликаты по heavy_sequence + light_sequence + antigen_sequence
# Оставляем первую запись из каждой группы (affinity отличается, но это не важно)

print(f"Размер до удаления дубликатов: {agab_df.shape}")

agab_df = agab_df.drop_duplicates(
    subset=['heavy_sequence', 'light_sequence', 'antigen_sequence'],
    keep='first'
)

print(f"Размер после удаления дубликатов: {agab_df.shape}")

Размер до удаления дубликатов: (324227, 11)
Размер после удаления дубликатов: (317611, 11)


In [ ]:
# Сохраняем датафрейм для использования в других ноутбуках
output_path = 'agab_filtered.parquet'
agab_df.to_parquet(output_path, index=False, engine='pyarrow')
print(f"Отфильтрованные данные сохранены в {output_path}")
print(f"Размер сохраненных данных: {agab_df.shape}")